In [2]:
using TensorKit
import TensorKit.×
include("./iPEPS_fSU2/iPEPS.jl")
include("./CTMRG_fSU2/CTMRG.jl")
@show pkgversion(TensorKit)

pkgversion(TensorKit) = v"0.12.2"


v"0.12.2"

In [ ]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = GradedSpace{fSU₂}((0 => 2), (1//2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 1), (1//2 => 1))
aspacet = GradedSpace{fSU₂}((0 => 1), (1//2 => 2))
aspacer = GradedSpace{fSU₂}((0 => 2), (1//2 => 2))
aspaceb = GradedSpace{fSU₂}((0 => 1), (1//2 => 1), (1 => 1))
# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

In [ ]:
a = id(pspace)
@tensor contractcheck=true b[] := a[x, in] * a[in, x]
@show scalar(b)

@show maximum(convert(Array, a))
@show minimum(convert(Array, a))
@show one(eltype(a))
@show one(mapreduce(eltype, promote_type, storagetype.(a)))
nothing

In [ ]:
t1 = 2 * id(pspace)
t2 = inv(sqrt(t1))
t3 = sqrt(inv(t1))
@assert t2 ≈ t3

In [ ]:
a = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
@show space(a)  # [l, t, p; r, b]
@show space(a') # [r, b; l, t, p]

permute!(a, a, ((1,2), (3,4,5)))
@show space(a)
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
A = id(pspace)
@show A
@tensor x = A[a, a]
@show x
nothing

In [ ]:
pspace = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacer = aspacel
aspacet = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspaceb = aspacet

Lx = 2
Ly = 2
Ms = Vector{TensorMap}(undef, Lx * Ly)
for ii in 1:Lx*Ly
    Ms[ii] = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
    Ms[ii] = Ms[ii] / norm(Ms[ii])
end
@assert Ms[1] != Ms[2]
ipeps = iPEPS(Ms, Lx, Ly)

envs = iPEPSenv(ipeps)
χ = 10
Nit = 2
nothing

In [ ]:
using JLD2

save("/home/tcmp2/JuliaProjects/testSave.jld2", "ipeps", ipeps, "envs", envs)

In [3]:
using JLD2
load("/home/tcmp2/JuliaProjects/testSave.jld2")
@show ipeps.Lx
@show envs.Ly
nothing

UndefVarError: UndefVarError: `ipeps` not defined

In [ ]:
@show numind(envs[1, 1].transfer.l)
@show numind(envs[1, 2].transfer.l)
envs[1, 1].transfer.l = id(pspace)
@show numind(envs[1, 1].transfer.l)
@show numind(envs[1, 1].transfer.r)
@show numind(envs[1, 1].transfer.t)
@show numind(envs[1, 1].transfer.b)
@show numind(envs[1, 2].transfer.l)
@assert envs[1, 1].corner.lt != envs[1, 2].corner.lt